In [1]:
%pip install python-dotenv  pandas  groq  Pinecone-client


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

doc =pd.read_csv('result.csv')
doc.head()

,job_url,title,company,location,date_posted,description
0,https://www.jobi.ai/jobs/full-stack-developer-...,full-stack-developer-postgres-django-vue-js---...,Mirus Solutions,"Bengaluru, Karnataka",2024-12-01,No job summary available.
1,https://career.luxoft.com/jobs/java-full-stack...,Java Full Stack Developer with React JS,Luxoft,"Bengaluru, Karnataka",2024-12-01,Project description\n\nLuxoft DXC Technology C...
2,https://jobs.gecareers.com/global/en/job/R3789...,Sr Software Engineer- Java FullStack,GE,"Bengaluru, Karnataka",2024-12-01,Job Description Summary\nResponsible for progr...
3,https://in.linkedin.com/jobs/view/java-net-ful...,Java/.Net Fullstack Developer,HARP,"Bengaluru, Karnataka",2024-11-29,Role - Java/ .Net Developer\n\nExp - 5+ Years\...
4,https://jobs.exxonmobil.com/ExxonMobil/job/Ben...,Surveillance & Optimization-Full stack developer,ExxonMobil,"Bengaluru, Karnataka",2024-11-28,"About us\n\nAt ExxonMobil, our vision is to le..."


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

# Step 1: Load Pre-trained Model and Tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Step 2: Load the CSV File
file_path = "result.csv"  # Update this to your file path
df = pd.read_csv(file_path)

# Ensure required columns exist
required_columns = ['title', 'description', 'company']
for column in required_columns:
    if column not in df.columns:
        raise ValueError(f"The '{column}' column is missing in the CSV file.")

# Step 3: Create a Combined Metadata Column
df['text'] = df['title'] + ": " + df['description'] + " " + df['company']

# Step 4: Define a Function to Generate BERT Embeddings
def generate_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean pooling of the last hidden state for sentence embedding
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
    return embeddings

# Step 5: Generate Embeddings for the Combined Column
df['embeddings'] = df['text'].apply(lambda x: generate_bert_embedding(x) if isinstance(x, str) else None)

# Step 6: Save the Updated DataFrame with Metadata and Embeddings
output_file = "result2.csv"
df.to_csv(output_file, index=False)
print(f"Updated CSV with metadata and embeddings saved as '{output_file}'.")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


Updated CSV with metadata and embeddings saved as 'result2.csv'.


In [4]:
re=pd.read_csv('result2.csv')
re.head()

,job_url,title,company,location,date_posted,description,text,embeddings
0,https://www.jobi.ai/jobs/full-stack-developer-...,full-stack-developer-postgres-django-vue-js---...,Mirus Solutions,"Bengaluru, Karnataka",2024-12-01,No job summary available.,full-stack-developer-postgres-django-vue-js---...,"[-0.23313391208648682, 0.0932946503162384, 0.2..."
1,https://career.luxoft.com/jobs/java-full-stack...,Java Full Stack Developer with React JS,Luxoft,"Bengaluru, Karnataka",2024-12-01,Project description\n\nLuxoft DXC Technology C...,Java Full Stack Developer with React JS: Proje...,"[-0.22971422970294952, 0.27602964639663696, 0...."
2,https://jobs.gecareers.com/global/en/job/R3789...,Sr Software Engineer- Java FullStack,GE,"Bengaluru, Karnataka",2024-12-01,Job Description Summary\nResponsible for progr...,Sr Software Engineer- Java FullStack: Job Desc...,"[-0.24388021230697632, 0.3478699326515198, 0.4..."
3,https://in.linkedin.com/jobs/view/java-net-ful...,Java/.Net Fullstack Developer,HARP,"Bengaluru, Karnataka",2024-11-29,Role - Java/ .Net Developer\n\nExp - 5+ Years\...,Java/.Net Fullstack Developer: Role - Java/ .N...,"[-0.35763758420944214, 0.06913603842258453, 0...."
4,https://jobs.exxonmobil.com/ExxonMobil/job/Ben...,Surveillance & Optimization-Full stack developer,ExxonMobil,"Bengaluru, Karnataka",2024-11-28,"About us\n\nAt ExxonMobil, our vision is to le...",Surveillance & Optimization-Full stack develop...,"[-0.1465856432914734, 0.171598419547081, 0.305..."


In [7]:

from pinecone import Pinecone, ServerlessSpec
import ast
pc = Pinecone(
    api_key='pcsk_2oqjji_6zGnJmu3WrdXATeNDYW8Z1YJYNLot7LHfJo4WRvwUzPAUfuQ6eoCAiGQZ1Dmcpw',
    # proxy_url='https://your-proxy.com',
    # proxy_headers=make_headers(proxy_basic_auth='username:password'),
    # ssl_ca_certs='path/to/cert-bundle.pem',
    ssl_verify=False
)
index_name = "job-data"
if index_name not in pc.list_indexes():
    # Create an index with dimensions matching your embeddings (e.g., 768 for BERT)
    pc.create_index(
        name=index_name,
        dimension=768,  # Adjust dimensions if necessary based on the embeddings you use
        metric="cosine",  # Use cosine similarity
        spec=ServerlessSpec(
            cloud="aws",  # Specify cloud if needed
            region="us-east-1"  # Specify region if needed
        )
    )
index = pc.Index(index_name)

# Step 3: Load the CSV File
file_path = "result2.csv" 
# uploade the csv file to the pinecone
df = pd.read_csv(file_path)

# Convert embeddings column from string representation to lists
df['embeddings'] = df['embeddings'].apply(ast.literal_eval)

# Step 4: Upsert Data into Pinecone
# Each vector needs a unique ID and optional metadata
for idx, row in df.iterrows():
    vector_id = str(idx)  # Use row index as the unique ID
    embedding = row['embeddings']  # The embedding vector
    metadata = {
        "title": row['title'],
        "description": row['description' ],
        "company": row['company'],
        "text": row['text'],
        "job_url": row.get('job_url') 

    }
    # Upsert the vector with metadata
    index.upsert([(vector_id, embedding, metadata)])

print("Database successfully connected and data uploaded to Pinecone.")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26

Database successfully connected and data uploaded to Pinecone.


In [ ]:
import pinecone
import os
import pandas as pd
from groq import Groq
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Pinecone initialization
pinecone = pinecone.Pinecone(api_key='pcsk_2oqjji_6zGnJmu3WrdXATeNDYW8Z1YJYNLot7LHfJo4WRvwUzPAUfuQ6eoCAiGQZ1Dmcpw')
index_name = "job-data"
index = pinecone.Index(index_name, host='https://job-data-iyg27yl.svc.aped-4627-b74a.pinecone.io')

# Groq initialization
groq_api_key = os.getenv("GROQ_API_KEY")
model_name = os.getenv("MODEL_NAME")
client = Groq(api_key=groq_api_key)

# Function to fetch all vector IDs from Pinecone
def fetch_all_vector_ids(index):
    vector_ids = []
    cursor = None

    while True:
        response = index.query(vector=[0] * 768, top_k=100, include_metadata=False, cursor=cursor)
        for match in response['matches']:
            vector_ids.append(match['id'])
        cursor = response.get('next_cursor')
        if not cursor:
            break

    return vector_ids


# Function to send data to the LLM and extract experience
def extract_experience(client, model_name, text, metadata):
    if not text:
        return 0

    # Combine text and metadata
    metadata_str = "\n".join([f"{key}: {value}" for key, value in metadata.items()])
    
    # Example-enhanced prompt
    prompt = f"""
Extract the years of experience required from the following job details. If experience is not mentioned explicitly, return 0.

### Examples:
- "Exp - 5+ Years" → 5
- "Must have 7 years of hands-on developer experience." → 7
- "6-8 years of relevant experience in software development in skills listed below." → 6 (use the minimum value if a range is given)

### Job Details:
{metadata_str}

### Job Text:
{text}
"""

    messages = [
        {"role": "system", "content": "You are a helpful assistant specializing in extracting years of experience from job details."},
        {"role": "user", "content": prompt},
    ]

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=0.5,
            max_tokens=100
        )
        experience = response["choices"][0]["message"]["content"].strip()
        return int(experience) if experience.isdigit() else 0
    except Exception as e:
        print(f"Error processing text:\n{text}\nError: {e}")
        return 0

# Fetch vector IDs from Pinecone
vector_ids = fetch_all_vector_ids(index)

# Fetch metadata and process each vector ID
results = []
for vector_id in vector_ids:
    response = index.fetch(ids=[vector_id])
    vector_data = response['vectors'].get(vector_id, {})
    metadata = vector_data.get('metadata', {})
    text = metadata.get('text', None)

    # Send data to LLM for experience extraction
    experience = extract_experience(client, model_name, text, metadata)

    # Add the extracted experience to the metadata
    metadata['experience'] = experience

    # Update Pinecone with the new metadata
    index.upsert([(vector_id, vector_data.get('values', [0] * 768), metadata)])

    results.append(metadata)
    print(f"Processing Vector ID: {vector_id}")
print(f"Job Text: {text}")
print(f"Metadata: {metadata}")

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Save the updated data with the 'experience' column
output_file = "pinecone_with_experience_updated.csv"
df.to_csv(output_file, index=False)

print(f"Updated data with experience column saved to '{output_file}'.")


Error processing text:
Founding Engineer (Full Stack): Skills:
Python (Programming Language), Java, AWS, JavaScript, React.js, AngularJS,

About The Role

We are setting up the core founding team of super talented and mission oriented engineers. As a Founding Engineer - FullStack, you will - (a) have an opportunity to significantly influence the overall technical strategy of the company, (b) own driving a complex product from the buy-in phase all the way through design, implementation & release, and (c) thoughtfully collaborate with multifunctional teams leading to value creation via tech. This role directly reports to the CTO.

As Founding Engineer, You Will
• Design system architecture and overall technical strategy for your product vertical
• Write and ship production code which would become foundation for the future
• Define domain specific data models, design and build backend APIs as well as web applications
• Manage end to end development, testing, deployment and monitoring of s

In [9]:
kk=pd.read_csv('pinecone_with_experience_updated.csv')
kk.head()

,company,description,job_url,text,title,experience
0,Thimblerr,"Skills:\nPython (Programming Language), Java, ...",https://in.linkedin.com/jobs/view/founding-eng...,Founding Engineer (Full Stack): Skills:\nPytho...,Founding Engineer (Full Stack),0
1,AUGUS IT SOLUTIONS,"Skills:\nTypescript, AWS, AWS SDK, GIT, React,...",https://in.linkedin.com/jobs/view/aws-full-sta...,"AWS Full Stack developer: Skills:\nTypescript,...",AWS Full Stack developer,0
2,DIATOZ: Digital A to Z Solutions,• Bachelor's degree in Computer Science.\n• 5+...,https://in.linkedin.com/jobs/view/java-fullsta...,Java Fullstack Developer (ReactJs) - 5+ years:...,Java Fullstack Developer (ReactJs) - 5+ years,0
3,Worldpanel by Kantar,Job Title: Senior Developer\n\nLocation: Banga...,https://in.linkedin.com/jobs/view/senior-devel...,Senior Developer (full stack): Job Title: Seni...,Senior Developer (full stack),0
4,EPAM Systems,EPAM is a leading global provider of digital p...,https://www.epam.com/careers/job-listings/job....,"Lead Java Full Stack Developer – Java 8, React...","Lead Java Full Stack Developer – Java 8, React...",0


In [ ]:
# Function to send data to the LLM and extract experience, also saving the LLM response
def extract_experience(client, model_name, text, metadata):
    if not text:
        return 0, "No description provided"  # Return a default value and message for no text

    # Combine text and metadata
    metadata_str = "\n".join([f"{key}: {value}" for key, value in metadata.items()])
    
    # Example-enhanced prompt
    prompt = f"""
Extract the years of experience required from the following job details. If experience is not mentioned explicitly, return 0.

### Examples:
1. "Exp - 5+ Years" → 5
2. "Must have 7 years of hands-on developer experience." → 7
3. "6-8 years of relevant experience in software development in skills listed below." → 6 (use the minimum value if a range is given)
4. "At least five years of experience in a managerial role." → 5
5. "Experience: None required." → 0

### Job Details:
{metadata_str}

### Job Text:
{text}
"""

    messages = [
        {"role": "system", "content": "You are a helpful assistant specializing in extracting years of experience from job details."},
        {"role": "user", "content": prompt},
    ]

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=0.5,
            max_tokens=100
        )
        
        # Log the LLM response for debugging
        print(f"LLM Response: {response}")
        
        experience = response["choices"][0]["message"]["content"].strip()
        llm_response = experience  # Save the LLM response
        return int(experience) if experience.isdigit() else 0, llm_response  # Return experience and response
    except Exception as e:
        print(f"Error processing text:\n{text}\nError: {e}")
        return 0, str(e)  # Return 0 and the error message

# Fetch vector IDs from Pinecone
vector_ids = fetch_all_vector_ids(index)

# Fetch metadata and process each vector ID
results = []
for vector_id in vector_ids:
    response = index.fetch(ids=[vector_id])
    vector_data = response['vectors'].get(vector_id, {})
    metadata = vector_data.get('metadata', {})
    text = metadata.get('text', None)

    # Send data to LLM for experience extraction and get LLM response
    experience, llm_response = extract_experience(client, model_name, text, metadata)

    # Add the extracted experience and LLM response to the metadata
    metadata['experience'] = experience
    metadata['llm_response'] = llm_response  # Save the LLM response in metadata

    # Update Pinecone with the new metadata
    index.upsert([(vector_id, vector_data.get('values', [0] * 768), metadata)])

    results.append(metadata)

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Save the updated data with the 'experience' and 'llm_response' columns
output_file = "pinecone_with_experience_and_llm_response.csv"
df.to_csv(output_file, index=False)

print(f"Updated data with experience and LLM response saved to '{output_file}'.")


LLM Response: ChatCompletion(id='chatcmpl-0261869a-daa4-4349-88e0-2d426c032728', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='0', role='assistant', function_call=None, tool_calls=None))], created=1733138273, model='llama-3.2-90b-vision-preview', object='chat.completion', system_fingerprint='fp_9c2a937c92', usage=CompletionUsage(completion_tokens=2, prompt_tokens=2428, total_tokens=2430, completion_time=0.0080453, prompt_time=0.322387656, queue_time=0.006012681999999991, total_time=0.330432956), x_groq={'id': 'req_01je3hemp4fzytj3he6n6qfgzf'})
Error processing text:
Software Engineer (Full Stack): Job Description Summary
Responsible for designing, building, delivering and maintaining software applications & services. Working in the areas of machine, cloud, platform and/or application. Responsible for software lifecycle including activities such as requirement analysis, documentation/procedures and implementation.

Job Description



In [11]:
gh=pd.read_csv('pinecone_with_experience_and_llm_response.csv')
gh.head()

,company,description,experience,job_url,text,title,llm_response
0,GE,Job Description Summary\nResponsible for desig...,0,https://jobs.gecareers.com/global/en/job/R3789...,Software Engineer (Full Stack): Job Descriptio...,Software Engineer (Full Stack),'ChatCompletion' object is not subscriptable
1,EPAM Systems,EPAM is a leading global provider of digital p...,0,https://www.epam.com/careers/job-listings/job....,"Java Full-Stack Developer – Java 8, ReactJS, M...","Java Full-Stack Developer – Java 8, ReactJS, M...",'ChatCompletion' object is not subscriptable
2,Thimblerr,"Skills:\nPython (Programming Language), Java, ...",0,https://in.linkedin.com/jobs/view/founding-eng...,Founding Engineer (Full Stack): Skills:\nPytho...,Founding Engineer (Full Stack),'ChatCompletion' object is not subscriptable
3,Capgemini,Choosing Capgemini means choosing a company wh...,0,https://careers.capgemini.com/job/Bangalore-Fu...,Full Stack Developer|4 TO 6 YEARS|BANGALORE: C...,Full Stack Developer|4 TO 6 YEARS|BANGALORE,'ChatCompletion' object is not subscriptable
4,Worldpanel by Kantar,Job Title: Senior Developer\n\nLocation: Banga...,0,https://in.linkedin.com/jobs/view/senior-devel...,Senior Developer (full stack): Job Title: Seni...,Senior Developer (full stack),'ChatCompletion' object is not subscriptable


In [12]:
import os
import time
import pandas as pd
from groq import Groq
from dotenv import load_dotenv
import pinecone

# Load environment variables
load_dotenv()

# Pinecone initialization
pinecone = pinecone.Pinecone(api_key='pcsk_2oqjji_6zGnJmu3WrdXATeNDYW8Z1YJYNLot7LHfJo4WRvwUzPAUfuQ6eoCAiGQZ1Dmcpw')
index_name = "job-data"
index = pinecone.Index(index_name, host='https://job-data-iyg27yl.svc.aped-4627-b74a.pinecone.io')

# Groq initialization
groq_api_key = os.getenv("GROQ_API_KEY")
model_name = os.getenv("MODEL_NAME")
client = Groq(api_key=groq_api_key)

# Function to fetch all vector IDs from Pinecone
def fetch_all_vector_ids(index):
    vector_ids = []
    cursor = None

    while True:
        response = index.query(vector=[0] * 768, top_k=100, include_metadata=False, cursor=cursor)
        for match in response['matches']:
            vector_ids.append(match['id'])
        cursor = response.get('next_cursor')
        if not cursor:
            break

    return vector_ids

# Function to send data to the LLM and extract experience, also saving the LLM response
def extract_experience(client, model_name, text, metadata):
    if not text:
        return 0, "No description provided"  # Return a default value and message for no text

    # Combine text and metadata for the prompt
    metadata_str = "\n".join([f"{key}: {value}" for key, value in metadata.items()])

    # Construct the prompt
    prompt = f"""
    Extract the years of experience required from the following job details. If experience is not mentioned explicitly, return 0.

    ### Examples:
    1. "Exp - 5+ Years" → 5
    2. "Must have 7 years of hands-on developer experience." → 7
    3. "6-8 years of relevant experience in software development in skills listed below." → 6 (use the minimum value if a range is given)
    4. "At least five years of experience in a managerial role." → 5
    5. "Experience: None required." → 0

    ### Job Details:
    {metadata_str}

    ### Job Text:
    {text}
    """

    messages = [
        {"role": "system", "content": "You are a helpful assistant specializing in extracting years of experience from job details."},
        {"role": "user", "content": prompt},
    ]

    try:
        # Make the request to the Groq API
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=0.5,
            max_tokens=100
        )
        
        # Extract the experience from the response
        experience = response.choices[0].message.content.strip()
        llm_response = experience  # Save the LLM response

        # Return experience (as integer) and LLM response
        return int(experience) if experience.isdigit() else 0, llm_response

    except Exception as e:
        print(f"Error processing text:\n{text}\nError: {e}")
        return 0, str(e)  # Return 0 and the error message

# Fetch vector IDs from Pinecone
vector_ids = fetch_all_vector_ids(index)

# Process each vector ID and update metadata
results = []
for vector_id in vector_ids:
    response = index.fetch(ids=[vector_id])
    vector_data = response['vectors'].get(vector_id, {})
    metadata = vector_data.get('metadata', {})
    text = metadata.get('text', None)

    # Send data to the LLM for experience extraction and get LLM response
    experience, llm_response = extract_experience(client, "llama3-70b-8192", text, metadata)

    # Update metadata with extracted experience and LLM response
    metadata['experience'] = experience
    metadata['llm_response'] = llm_response  # Save the LLM response in metadata

    # Update Pinecone with the new metadata one by one
    index.upsert([(vector_id, vector_data.get('values', [0] * 768), metadata)])

    # Append the result to the list
    results.append(metadata)

    # Optional: Add a delay to avoid hitting rate limits (e.g., 0.1 seconds)
    time.sleep(0.1)

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Save the updated data with the 'experience' and 'llm_response' columns
output_file = "pinecone_with_experience_and_llm_response.csv"
df.to_csv(output_file, index=False)

print(f"Updated data with experience and LLM response saved to '{output_file}'.")


Error processing text:
Surveillance & Optimization-Full stack developer: About us

At ExxonMobil, our vision is to lead in energy innovations that advance modern living and a net-zero future. As one of the world’s largest publicly traded energy and chemical companies, we are powered by a unique and diverse workforce fueled by the pride in what we do and what we stand for.

The success of our Upstream, Product Solutions and Low Carbon Solutions businesses is the result of the talent, curiosity and drive of our people. They bring solutions every day to optimize our strategy in energy, chemicals, lubricants and lower-emissions technologies.

We invite you to bring your ideas to ExxonMobil to help create sustainable solutions that improve quality of life and meet society’s evolving needs. Learn more about our What and our Why and how we can work together.

ExxonMobil’s affiliates in India

ExxonMobil’s affiliates have offices in India in Bengaluru, Mumbai and the National Capital Region.



In [2]:
import pandas as pd
sda=pd.read_csv('pinecone_with_experience_and_llm_response.csv')
sda.head()

,company,description,experience,job_url,llm_response,text,title
0,GE,Job Description Summary\nResponsible for progr...,0,https://jobs.gecareers.com/global/en/job/R3789...,The years of experience required for this job ...,Sr Software Engineer- Java FullStack: Job Desc...,Sr Software Engineer- Java FullStack
1,Worldpanel by Kantar,Job Title: Senior Developer\n\nLocation: Banga...,0,https://in.linkedin.com/jobs/view/senior-devel...,The years of experience required for this job ...,Senior Developer (full stack): Job Title: Seni...,Senior Developer (full stack)
2,Hexaware Technologies,"Primary Skill (C#, Net, Winforms, SQL)\n\nLoca...",0,https://in.linkedin.com/jobs/view/microsoft-fu...,The years of experience required for this job ...,Microsoft Full Stack Engineer - Professional: ...,Microsoft Full Stack Engineer - Professional
3,AUGUS IT SOLUTIONS,"Skills:\nTypescript, AWS, AWS SDK, GIT, React,...",0,https://in.linkedin.com/jobs/view/aws-full-sta...,The years of experience required for this job ...,"AWS Full Stack developer: Skills:\nTypescript,...",AWS Full Stack developer
4,EPAM Systems,EPAM is a leading global provider of digital p...,0,https://www.epam.com/careers/job-listings/job....,The required years of experience for this job ...,"Lead Java Full Stack Developer – Java 8, React...","Lead Java Full Stack Developer – Java 8, React..."
